# Vowpal Wabbit

Поработаем с той же задачей и с теми же данными, на которых мы настраивали параметры xgboost.

Посмотрим, справятся ли линейные алгоритмы, реализованные в vowpal wabbit, с этой задачей.

In [ ]:
import pandas as pd

X_train = pd.read_csv('train_medium.csv')
X_test = pd.read_csv('test_medium.csv')

Измените функцию save_to_vw из предыдущего ноутбука (см.ниже) так, чтобы она записывала файлы train_medium и test_medium в нужном формате. Не забудьте, что столбец 'Disbursed' является целевым, поэтому а) он определяет значение переменной target, b) его не нужно записывать в файл-матрицу с признаками.

In [ ]:
import re

def save_to_vw(data, fname):
    with open(fname, 'w') as fout:
        for _, row in data.iterrows():
            text = filter(lambda x: len(x) > 1, re.split("[^a-z]",
                                    row.BodyMarkdown.lower()))
            text = ' '.join(text)
            if row.OpenStatus == "open":
                target = 1
            else:
                target = -1
            fout.write('{0} |n 0:{1} {2} |t {3}\n'.format(target, 
                                        row.ReputationAtPostCreation,
                                        row.Tag1,
                                        text))

In [ ]:
save_to_vw(X_train, 'train_medium_vw.vw')
save_to_vw(X_test, 'test_medium_vw.vw')

Скопируйте полученные файлы в папку share и обучите vowpal wabbit. Например, предсказания классов можно сохранять в файл с именем preds.txt, а предсказания вероятностей - в файл probs.txt.

Поэкспериментируйте с параметрами vowpal wabbit и постарайтесь добиться наилучшего качества на тесте.

Постарайтесь, насколько это возможно, приблизить ROC-AUC к значению, полученному после оптимизации xgboost.

In [ ]:
from sklearn import metrics

probs = []
f=open('probs.txt','r')
lines = f.readlines()
for line in lines:
    probs.append(float(line))
    
print 'ROC-AUC:', metrics.roc_auc_score(X_test['Disbursed'], probs)

In [ ]:
res = []
f=open('preds.txt','r')
lines = f.readlines()
for line in lines:
    if float(line) < 0:
        res.append(0)
    else:
        res.append(1)
    
print 'accuracy:', metrics.accuracy_score(X_test['Disbursed'].values, res)